In [ ]:
# pip install -U pymoo

In [ ]:
import numpy as np
import pandas as pd
import json

from data_preprocess import *
from run_ga import *
import matplotlib.pyplot as plt
import pickle

In [ ]:
tet_path = "./data/tet.json"
fdr_path = "./data/all_class_mutation/Chart-1_fdr.json"

with open(tet_path, 'r') as f:
    tet_data = json.load(f)


with open(fdr_path, 'r') as f:
    fdr_data = json.load(f)

chart_1_tet = tet_data['Chart-1']
chart_1_fdr = dict()

for key, value in fdr_data['Chart_1'].items():
    chart_1_fdr[key] = value['mutation-score']


chart_1_tet = sorted(chart_1_tet.items())
chart_1_fdr = sorted(chart_1_fdr.items())

In [ ]:
tests = [i[0] for i in chart_1_tet]
execution_times = np.array([i[1] for i in chart_1_tet])
fault_detections = np.array([i[1] for i in chart_1_fdr])

test_cases = np.column_stack((execution_times, fault_detections))
adequacy_scores = get_adequacy_scores(fault_detections)

In [ ]:
bitflip = run_nsga(test_cases, False)

In [ ]:
adeq = run_nsga_with_adequecy(test_cases, adequacy_scores, False)

In [ ]:
bitflip_res = np.abs(bitflip.F)
adeq_res = np.abs(adeq.F)

plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
plt.xlabel("TET")
plt.ylabel("FDR")
plt.legend()

In [ ]:
with open('data/experiment_result/f_fdr_m_fdr/Chart_1_bitflip.pkl', 'rb') as f:
    bitflip = pickle.load(f)

with open('data/experiment_result/f_fdr_m_fdr/Chart_1_adeq.pkl', 'rb') as f:
    adeq = pickle.load(f)
# print(data.F)

bitflip_res = bitflip_res = np.abs(bitflip.F)
adeq_res = np.abs(adeq.F)

plt.scatter(bitflip_res[:, 0], bitflip_res[:, 1], label="Bitflip")
plt.scatter(adeq_res[:, 0], adeq_res[:, 1], label="Adequacy")
plt.xlabel("TET")
plt.ylabel("Effectiveness")
plt.legend()

In [ ]:
types = ["f_cov_m_fdr", "f_fdr_m_fdr"]
tet_dict = {}
for type in types:
    for target in os.listdir('./data/merged_data'):
        pid, vid = os.path.splitext(target)[0].split('_')

        with open(f'data/experiment_result/{type}/{pid}_{vid}_bitflip.pkl', 'rb') as f:
            bitflip = pickle.load(f)

        with open(f'data/experiment_result/{type}/{pid}_{vid}_adeq.pkl', 'rb') as f:
            adeq = pickle.load(f)

        with open(f'data/merged_data/{pid}_{vid}.pkl', 'rb') as f:
            data = pickle.load(f)

        with open(f"data/failing_tests/{pid}-{vid}/failing_tests", "r") as f:
            failing_tests = f.readlines()

        min_id_adeq = None
        for i, sol in enumerate(adeq.X):
            # print(data.index[sol].shape)
            selected_tests = data.index[sol]
            if all(test in selected_tests for test in failing_tests):
                tet = adeq.F[i, 0]
                if min_id_adeq == None:
                    min_id_adeq = i
                elif tet < adeq.F[min_id_adeq, 0]:
                    min_id_adeq = i

        min_id_bitflip = None
        for i, sol in enumerate(bitflip.X):
            # print(data.index[sol].shape)
            selected_tests = data.index[sol]
            if all(test in selected_tests for test in failing_tests):
                tet = bitflip.F[i, 0]
                if min_id_bitflip == None:
                    min_id_bitflip = i
                elif tet < bitflip.F[min_id_bitflip, 0]:
                    min_id_bitflip = i

        # print("Adeq tet")
        # print(adeq.F[min_id_adeq, 0])
        # print("Bitflip tet")
        # print(bitflip.F[min_id_bitflip, 0])
        # print("\n")
        tet_dict[f"{pid}_{vid}"] = {"adeq": adeq.F[min_id_adeq, 0], "bigflip": bitflip.F[min_id_bitflip, 0]}
    # with
    print(type)
    print(tet_dict)
    with open(f"./data/experiment_result/{type}/min_tet_with_failing_tests.json", 'w') as f:
        json.dump(tet_dict, f)
